In [ ]:
import librosa   
import IPython.display as ipd
import matplotlib.pyplot as plt
import os
import soundfile as sf
import numpy as np
from scipy.io import wavfile #for audio processing
import warnings
warnings.filterwarnings("ignore")

In [58]:
import os
os.getcwd()


'/home/prateek/conversational_bot/speech_recognition'

In [59]:
path = '/sample_data'
os.chdir(os.getcwd()+path )
os.getcwd()

'/home/prateek/conversational_bot/speech_recognition/sample_data'

In [60]:
os.listdir()

['227', '198']

In [61]:
list_IDs = []

for direc in os.listdir():
        file = [ f for f in os.listdir(os.getcwd() + '/' + direc ) if f.endswith('.flac')]
        for f in file:
            list_IDs.append(direc + '/' + f)

print("string :",len(list_IDs))
print((list_IDs[0]),"\n",(list_IDs[110]))

string : 111
227/19-227-0021.flac 
 198/19-198-0001.flac


In [62]:
import numpy as np

#%%
# From Baidu ba-dls-deepspeech - https://github.com/baidu-research/ba-dls-deepspeech
# Character map list

char_map_str = """
<SPACE> 0
a 1
b 2
c 3
d 4
e 5
f 6
g 7
h 8
i 9
j 10
k 11
l 12
m 13
n 14
o 15
p 16
q 17
r 18
s 19
t 20
u 21
v 22
w 23
x 24
y 25
z 26
' 27
"""

char_map = {}
index_map = {}

for line in char_map_str.strip().split('\n'):
    ch, index = line.split()
    char_map[ch] = int(index)
    index_map[int(index)] = ch

index_map[0] = ' '


def get_param(Y):
  labels = np.zeros((520,23))

  for idx,label in enumerate(Y):
    new = []
    for word in label.split():
      for c in word:
        if c not in char_map:
          continue
        elif c == "'":
          continue
        else:
          ch = char_map[c]
          new.append(ch)
#       if(len(new) < max_label):
#         new.append(0)
    while(len(new) < max_label):
      new.append(27)
    labels[idx,:] = np.array(new)

  input_length = np.array([max_label for _ in Y])
  label_length = np.array([max_label for _ in Y])


  return labels , input_length , label_length

In [63]:
char_map

{'<SPACE>': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 "'": 27}

In [64]:
import tensorflow
from tensorflow import keras as keras

In [66]:
class AudioDataGenerator(keras.utils.Sequence):
  #batch_size, character index, max string length, 
  def __init__(self, source_dir, char_idx, batch_size=100, target_shape=(101, 198), max_string_len=10, augment=False, shuffle=True):
    self.data_dir = source_dir
    self.char_idx = char_idx
    self.batch_size = batch_size
    self.max_string_len = max_string_len
    self.augment = augment
    self.dims = target_shape
    self.shuffle = shuffle

    #Count number of training files
    self.total_examples = 0
    self.list_IDs = []

    self.dir_list = os.listdir(self.data_dir)
    for i, dir in enumerate(self.dir_list):
        example_list = os.listdir(self.data_dir +'/'+ dir)
        example_IDs = [[i, e] for e in example_list]
        num_examples = len(example_list)
      
        self.total_examples += num_examples
        self.list_IDs += example_IDs

    print ("found " + str(self.total_examples) + " examples")

    #call on_epcoh_end to perpare initial data indexes
    self.on_epoch_end()

  def __len__(self):
    #number of training steps per epoch
    return int(np.floor(self.total_examples/self.batch_size))
    #return 100

  def __getitem__(self, index):
    # Generate indexes of the batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    list_IDs_temp = [self.list_IDs[k] for k in indexes]

    # Generate data
    X, y = self.__data_generation(list_IDs_temp)

    #for CTC you to give 4 arguments, I have only given inputs and labels
    #input shape and label shape should also be output by this function
#     input_length = np.array([self.total_timedistributed_output for _ in range(self.batch_size)])
#     label_length = np.array([self.max_label for _ in range(self.batch_size)])
        
    input_shape = np.zeros([self.batch_size, 1])
    label_length = np.zeros([self.batch_size, 1])
    input_shape[:] = 101
    label_length[:] = self.max_string_len
    inputs = {"the_inputs": X, "the_labels": y, "input_length":input_shape, "label_length": label_length}
    outputs = {"ctc": y}
    return (inputs, outputs)

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
        np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp):
    # Initialization
    X = np.empty((self.batch_size, *self.dims))
    y = np.empty((self.batch_size, self.max_string_len), dtype=int)

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
        wav_file = self.data_dir + self.dir_list[ID[0]] + "/" + ID[1]
        spec_gram = self.__graph_spectrogram(wav_file)
      
      #I have fixed the dimensions, you will need to change this
    if (spec_gram.shape == (101, 198)):
        X[i,] = spec_gram
    else:
        X[i,] = np.zeros((101, 198))

    y[i,] = self.__str_to_label(self.dir_list[ID[0]])

    return X, y

  def __graph_spectrogram(self, flac_file):
    rate, data = sf.read(flac_file)
    nfft = 200 # Length of each window segment
    fs = 8000  #Sampling frequencies
    noverlap = 120 # Overlap between windows
    nchannels = data.ndim
    if nchannels == 1:
        pxx, freqs, bins, im = plt.specgram(data, nfft, fs, noverlap = noverlap)
    elif nchannels == 2:
        pxx, freqs, bins, im = plt.specgram(data[:,0], nfft, fs, noverlap = noverlap)
    return pxx

  #get labels from input word

  def __str_to_label(self, word):
    #array of length max string length, 0 padded (actually it should be blank token :p)
    label = np.zeros((self.max_string_len), dtype=int)
    for i in range(len(word)):
      label[i] = int(self.char_idx[word[i]])

    return label

In [67]:
from keras.layers import *
from keras.layers.wrappers import TimeDistributed
from keras.layers.merge import Add
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from keras.utils import plot_model

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args    
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length )    
    
        
#%%
class model():
    """
    Usage:
        sr_ctc = CTC(enter input_size and output_size)
        sr_ctc.build()
        sr_ctc.m.compile()
        sr_ctc.tm.compile()
    """       
    def __init__(self,
                 input_size=None, 
                 output_size=None,
                 initializer='glorot_uniform'):
        self.input_size = input_size
        self.output_size = output_size
        self.initializer = initializer
        self.m = None
        self.tm = None
                   
    def build(self, 
              conv_filters = 200,
              conv2d_filters = 13,
              conv_size = 5,
              conv2d_strides = 1,
              conv_strides = 4,
              act = 'relu',
              rnn_layers = 2,
              LSTM_units = 128,
              drop_out = 0.8):
           
        i = Input(shape = self.input_size, name = 'input')
        x = Conv2D(conv2d_filters,
                   conv_size,
                   strides = (conv2d_strides, conv2d_strides),
                   padding = "same",
                   name = "conv2d1")(i)
        x = BatchNormalization()(x)
        x = Activation(act)(x)
        x = Conv2D(conv2d_filters,
                   conv_size,
                   strides = (conv2d_strides , conv2d_strides),
                   padding = "same",
                   name = "conv2d2")(x)
        x = BatchNormalization()(x)
        x = Activation(act)(x)
        x = Reshape([606,-1])(x)
        x = Conv1D(conv_filters, 
                   conv_size, 
                   strides = conv_strides,
                   padding = "same", 
                   name = 'conv1d1')(x)
        x = BatchNormalization()(x)
        x = Activation(act)(x)
        x = Conv1D(conv_filters, 
                   conv_size, 
                   strides = conv_strides,
                   padding = "same", 
                   name = 'conv1d2')(x)
        x = BatchNormalization()(x)
        x = Activation(act)(x)
        x = Reshape([200,38])(x)
        x = Dense(23 , activation="softmax", name ="dense")(x)
        x = Reshape([23,-1])(x)
        for _ in range(rnn_layers):          
            x = Bidirectional(LSTM(LSTM_units, 
                                   return_sequences = True))(x)
            x = Dropout(drop_out)(x)
            x = BatchNormalization()(x)
        y_pred = TimeDistributed(Dense(self.output_size, 
                                       activation = 'softmax'))(x)        
        # ctc inputs
        labels = Input(name='the_labels', shape=[None,], dtype='int32')
        input_length = Input(name='input_length', shape=[1], dtype='int32')
        label_length = Input(name='label_length', shape=[1], dtype='int32')    
#        label_length = np.array([self.max_label for _ in range(self.batch_size)])
#         label_length=150
        loss_out = Lambda(ctc_lambda_func, 
                          output_shape=(1,), 
                          name='ctc')([y_pred,
                                        labels,
                                        input_length,
                                        label_length])        
        self.tm = Model(inputs = i,
                        outputs = y_pred)
        self.m = Model(inputs = [i, 
                                 labels, 
                                 input_length, 
                                 label_length], 
                        outputs = loss_out)
        return self.m, self.tm

In [68]:
training_generator = AudioDataGenerator( os.getcwd(), char_map)
training_generator.__getitem__(3)

found 113 examples


UnboundLocalError: local variable 'spec_gram' referenced before assignment

In [69]:
def ctc(y_true, y_pred):
    return y_pred

In [70]:
sr_ctc = model((101,198,1), 28)
sr_ctc.build()

(<keras.engine.training.Model at 0x7ff371b961d0>,
 <keras.engine.training.Model at 0x7ff371bc4be0>)

In [71]:
sr_ctc.m.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 101, 198, 1)  0                                            
__________________________________________________________________________________________________
conv2d1 (Conv2D)                (None, 101, 198, 13) 338         input[0][0]                      
__________________________________________________________________________________________________
batch_normalization_55 (BatchNo (None, 101, 198, 13) 52          conv2d1[0][0]                    
__________________________________________________________________________________________________
activation_37 (Activation)      (None, 101, 198, 13) 0           batch_normalization_55[0][0]     
____________________________________________________________________________________________

In [72]:
sr_ctc.m.compile(loss = ctc, optimizer = 'adam', metrics = ['accuracy'])
sr_ctc.tm.compile(loss = ctc, optimizer = 'adam')

In [ ]:
history = sr_ctc.m.fit_generator(training_generator, epochs=1)